# Procesamiento y detección de puntos de referencia faciales

Este notebook tiene como objetivo procesar una carpeta de imágenes para:

1. Detectar puntos clave del rostro (como nariz) usando la API de visión de OpenAI.
2. Modificar el prompt para que se identifiquen dichos puntos con coordenadas exactas.
3. Bajar la resolución de las imágenes para acelerar el procesamiento y reducir costos.
4. Ordenar las imágenes de forma consistente
5. Recortar las imágenes centradas en los puntos clave detectados para una mejor visualización o análisis posterior en el pdf auto generado.

Cada paso está debidamente documentado con funciones específicas.

https://platform.openai.com/docs/guides/images-vision?api-mode=responses

## 0. Configuración inicial
Antes de comenzar con el procesamiento de imágenes, es necesario configurar el entorno de trabajo:

- Se importan las librerías necesarias, tanto para manejo de archivos, imágenes y conexión con la API de OpenAI.
- Se cargan las variables de entorno desde un archivo `.env`, donde debe estar la clave de API (`OPENAI_API_KEY`) y cualquier otro parámetro sensible.
- Se inicializa el cliente de OpenAI.
- Se definen los directorios de entrada (`SRC_DIR`) y salida (`DST_DIR`) donde se encuentran las imágenes originales y donde se guardarán las imágenes redimensionadas.
- Se establece la resolución objetivo (`TARGET_WIDTH`) para estandarizar el tamaño de las imágenes.

Este paso garantiza que el flujo posterior (resize, codificación, análisis con la API, etc.) se pueda ejecutar de forma ordenada y reproducible.


In [ ]:
# --- Librerías necesarias ---
import os
import time
import glob
import base64
import json

from dotenv import load_dotenv
from openai import OpenAI

import pandas as pd
from PIL import Image, ImageOps, ImageDraw

from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm

import ast

In [13]:
# --- Cargar variables de entorno ---
load_dotenv()

# --- Inicializar cliente OpenAI ---
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)
OPENAI_ASSISTANT_ID = os.getenv("OPENAI_ASSISTANT_ID")  # si lo vas a usar más adelante

# --- Directorios de trabajo ---
SRC_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/sineditar"
DST_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized"
os.makedirs(DST_DIR, exist_ok=True)

# --- Parámetros de redimensionado ---
TARGET_WIDTH = 1350
valid_extensions = ('.jpg', '.jpeg')  # extensiones válidas

In [14]:
def resize_image(src_path, dst_path, target_width):
    """
    Redimensiona una imagen a un ancho específico manteniendo la proporción.

    Args:
        src_path (str): Ruta de la imagen original.
        dst_path (str): Ruta donde se guardará la imagen redimensionada.
        target_width (int): Ancho deseado en píxeles.
    """
    img = Image.open(src_path)
    img = ImageOps.exif_transpose(img)  # corrige orientación según EXIF
    w_percent = target_width / img.width
    target_height = int(img.height * w_percent)
    img_resized = img.resize((target_width, target_height), Image.LANCZOS)
    img_resized.save(dst_path)
    img.close()

In [15]:
def encode_image_to_base64(image_path):
    """
    Codifica una imagen en base64 para usarla en la API de OpenAI.

    Args:
        image_path (str): Ruta de la imagen.

    Returns:
        str: Cadena codificada en base64.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [16]:
# Iterar sobre las imágenes del directorio y aplicar resize
for img_path in glob.glob(os.path.join(SRC_DIR, "*")):
    if img_path.lower().endswith(valid_extensions):
        filename = os.path.basename(img_path)
        dst_path = os.path.join(DST_DIR, filename)
        resize_image(img_path, dst_path, TARGET_WIDTH)

## 2. Procesamiento de imágenes con la API de OpenAI

En este paso se envían las imágenes redimensionadas a la API de OpenAI para:

- Detectar la cantidad de ojos visibles.
- Determinar la posición de la cabeza y la expresión facial.
- Obtener coordenadas de la punta de la nariz.
- Evaluar la calidad de la imagen (calificación entre 0 y 1).

Se utiliza un prompt estructurado que pide a la API responder en formato JSON estandarizado.  
Todos los resultados se almacenan en una lista y posteriormente en un DataFrame para su análisis o exportación.

Este paso es clave para el análisis facial automatizado.

In [18]:
# Prompt para clasificación facial y puntos clave
prompt = """
Recibe una imagen de un sujeto y realiza lo siguiente:

0. Ojos visibles:
   - Analiza cuántos ojos se aprecian:
     * Si hay 2 ojos visibles ➔ sigue al paso 1
     * Si hay 1 ojo visible ➔ posición = “Perfil izquierda” si mira a la izquierda desde la perspectiva de nosotros o 
        “Perfil derecha” si mira a la derecha desde la perspectiva de nosotros 
     * NA (no se puede determinar o no es la foto del rostro de una persona)

1. Clasificación facial (solo si hay 2 ojos o no aplicó paso 0):
   - Posición de la cabeza (elige UNA):
     * Frente (rostro frontal, ambos ojos simétricos y visibles)
     * Girado izquierda (ambos ojos visibles, cabeza girada hacia la izquierda, >50% del rostro visible)
     * Girado derecha (ambos ojos visibles, cabeza girada hacia la derecha, >50% del rostro visible)
     * NA (no se puede determinar o no es la foto del rostro de una persona)

   - Expresión facial (elige UNA):
     * Sin dentadura (boca cerrada o apenas entreabierta sin dientes visibles)
     * Con dentadura (dentadura visible claramente)
     * NA (no se puede determinar)

2. Detección de puntos clave:
   - nariz: coordenada en pixeles [x, y] de la punta de la nariz, o "NA" si no es detectable
   - calificación: número real entre 0 y 1 que evalúa la calidad de la imagen

Formato de salida ÚNICAMENTE un array JSON con un objeto así:

[
  {
    "nombre_archivo": "IMG_1234.JPG",
    "posicion": "Perfil izquierda",
    "expresion": "Sin dentadura",
    "nariz": [150, 550],
    "calificacion": 0.85
  }
]
"""

In [19]:
# Lista para guardar los resultados
results = []

# Iterar sobre las imágenes redimensionadas
for filename in os.listdir(DST_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(DST_DIR, filename)
        image_base64 = encode_image_to_base64(image_path)

        # Construir el input para el modelo
        input_data = [
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": f"{prompt}\nImagen: {filename}"},
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{image_base64}"
                    }
                ]
            }
        ]

        # Hacer la solicitud a la API
        response = client.responses.create(
            model="o4-mini",  # o el modelo que estés usando
            reasoning={"effort": "high"},
            input=input_data
        )

        # Extraer la respuesta
        if hasattr(response, 'output'):
            result = response.output[1].content[0].text  # Ajusta esto si cambia el formato

            try:
                # Convertir string a objeto Python
                data = eval(result) if isinstance(result, str) else result

                if isinstance(data, dict):
                    results.append(data)
                elif isinstance(data, list):
                    results.extend(data)

            except Exception as e:
                print(f"Error procesando la respuesta de {filename}: {e}")

In [24]:
# Crear DataFrame con resultados
df = pd.DataFrame(results)

# Guardar en CSV
df.to_csv('/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data_01.csv', index=False)

## 3. Recorte de Imágenes

En esta sección se realiza el recorte de las imágenes originales con el objetivo de centrar el rostro del paciente utilizando como referencia la posición de la nariz. El recorte se hace de manera dinámica, adaptado a cada tipo de pose (frontal, perfil, girado, etc.).

### Variables utilizadas para el recorte

- `left`, `right`: determinan el ancho del recorte a partir del centro de la nariz.
- `top`, `bottom`: determinan el alto del recorte hacia arriba y hacia abajo desde la nariz.

Estas variables están definidas de forma personalizada por tipo de pose en el diccionario `recortes_por_posicion`.

### Relación de aspecto

Se busca mantener una relación de aspecto uniforme en las imágenes resultantes, cercana a **3:4 (ancho:alto)**. Para conseguirlo:

- Se mantiene constante el alto del recorte (`top + bottom`, aproximadamente 2350 px).
- Se ajustan los valores de `left` y `right` para lograr un ancho total de aproximadamente 1760 px.

Con esto, se garantiza que las imágenes mantengan una proporción visual coherente y que el rostro del paciente siempre quede bien centrado, sin perder las zonas de interés facial.


In [ ]:
# --- Configuración ---
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/sineditar"
output_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/fotos_finales"
resized_width_csv = 1350

recortes_por_posicion = {
    "Perfil izquierda":    {"left": 400, "right": 1360, "top": 1400, "bottom": 950},
    "Girado izquierda":    {"left": 600, "right": 1160, "top": 1400, "bottom": 950},
    "Frente":              {"left": 880, "right": 880,  "top": 1400, "bottom": 950},
    "Girado derecha":      {"left": 1160, "right": 600, "top": 1400, "bottom": 950},
    "Perfil derecha":      {"left": 1360, "right": 400, "top": 1400, "bottom": 950}
}

posiciones_deseadas = list(recortes_por_posicion.keys())
expresiones_deseadas = ["Sin dentadura", "Con dentadura"]

In [ ]:
# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer CSV
df = pd.read_csv(csv_path)

# --- Función de recorte dinámico ---
def crop_relative_to_nose(img, nose_coord, l, r, t, b):
    x, y = nose_coord
    left = max(x - l, 0)
    upper = max(y - t, 0)
    right = min(x + r, img.width)
    lower = min(y + b, img.height)
    return img.crop((left, upper, right, lower))

# --- Procesamiento principal ---
for expresion in expresiones_deseadas:
    for pos in posiciones_deseadas:
        subset = df[(df['expresion'] == expresion) & (df['posicion'] == pos)]
        if not subset.empty:
            row = subset.iloc[0]
            nombre = row['nombre_archivo']
            nariz_resized = ast.literal_eval(row['nariz'])
            src_path = os.path.join(img_folder, nombre)

            if os.path.exists(src_path):
                try:
                    img = Image.open(src_path)
                    img = ImageOps.exif_transpose(img)

                    # Escalar coordenadas
                    scale_factor = img.width / resized_width_csv
                    nariz_original = (
                        int(nariz_resized[0] * scale_factor),
                        int(nariz_resized[1] * scale_factor)
                    )

                    # Obtener márgenes personalizados
                    recorte = recortes_por_posicion[pos]
                    img_cropped = crop_relative_to_nose(
                        img,
                        nariz_original,
                        recorte["left"],
                        recorte["right"],
                        recorte["top"],
                        recorte["bottom"]
                    )

                    # Guardar
                    nombre_base = f"cropped_{expresion.replace(' ', '_')}_{nombre}"
                    out_path = os.path.join(output_folder, nombre_base)
                    img_cropped.save(out_path)
                    print(f"Recortada: {nombre} | {pos} | Expresión: {expresion}")
                except Exception as e:
                    print(f"Error con {nombre}: {e}")
            else:
                print(f"No encontrada: {nombre}")



## 4. Generación de PDF ordenado con imágenes

En este paso se genera un PDF con las imágenes organizadas de forma visualmente clara para su presentación.

### Estructura del PDF:
- **Página 1 (vertical)**: Imagen frontal del sujeto con dentadura, centrada y acompañada de datos como nombre, edad y fecha.
- **Página 2 (horizontal)**: 2 filas de imágenes:
  - Fila superior: imágenes con expresión *sin dentadura* en orden de posición (perfil, girado, frente...).
  - Fila inferior: imágenes *con dentadura* en el mismo orden.

Este formato ayuda a mostrar la variedad de tomas faciales de manera sistemática.


In [51]:
# --- Rutas ---
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/fotos_finales"
output_pdf = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/output_fotos_finales.pdf"

# Leer CSV
df = pd.read_csv(csv_path)

In [52]:
# === PÁGINA 1 ===
c = canvas.Canvas(output_pdf, pagesize=A4)
w, h = A4

frontal = df[(df['posicion'] == 'Frente') & (df['expresion'] == 'Con dentadura')].iloc[0]
filename = f"cropped_Con_dentadura_{frontal['nombre_archivo']}"
img_path = os.path.join(img_folder, filename)

img = Image.open(img_path)
if img.width > img.height:
    img = img.rotate(90, expand=True)
    img.save(img_path)

# Ajustar tamaño manteniendo aspecto
img_width = 12 * cm
aspect = img.height / img.width
img_height = img_width * aspect

# Coordenadas centradas
x_img = (w - img_width) / 2
y_img = h - img_height - 6*cm

# Dibujar sombra DETRÁS (como marco expandido)
shadow_margin = 0.2 * cm
c.saveState()
c.setFillColor(Color(0, 0, 0, alpha=0.2))  # sombra negra 20%
c.rect(x_img - shadow_margin,
       y_img - shadow_margin,
       img_width + 2 * shadow_margin,
       img_height + 2 * shadow_margin,
       fill=1, stroke=0)
c.restoreState()

# Dibujar imagen encima
c.drawImage(img_path, x_img, y_img, width=img_width, height=img_height)

# Texto
c.setFont("Helvetica", 16)
c.setFillColor(black)
c.drawCentredString(w / 2, y_img - 1.5*cm, "SEBASTIAN MORA AGUILERA")
c.setFont("Helvetica", 12)
c.drawString(2*cm, y_img - 3*cm, "EDAD: 10 AÑOS")
c.drawString(8*cm, y_img - 3*cm, "FECHA DE NACIMIENTO: 08/03/2015")
c.drawString(2*cm, y_img - 3.8*cm, "FECHA DE TOMA: 17/05/2025")
c.drawString(2*cm, y_img - 4.6*cm, "DRA. CLAUDIA RIVERO MARIN")
c.showPage()

In [53]:
# === PÁGINA 2 ===
c.setPageSize(landscape(A4))
w, h = landscape(A4)

orden = ["Perfil izquierda", "Girado izquierda", "Frente", "Girado derecha", "Perfil derecha"]
sin_dent = df[df['expresion'] == "Sin dentadura"].copy()
con_dent = df[df['expresion'] == "Con dentadura"].copy()

def get_ordered(filtrado, expresion):
    resultado = []
    for pos in orden:
        match = filtrado[filtrado['posicion'] == pos]
        if not match.empty:
            nombre = match.iloc[0]['nombre_archivo']
            archivo = f"cropped_{expresion.replace(' ', '_')}_{nombre}"
            resultado.append(archivo)
    return resultado

row1 = get_ordered(sin_dent, "Sin dentadura")
row2 = get_ordered(con_dent, "Con dentadura")

# Ajustar tamaño y separación para que entren bien
num_imgs = len(orden)
margin_horizontal = 1.5 * cm
available_width = w - 2 * margin_horizontal
max_img_width = 5.5 * cm  # reducir si se salen
gap = (available_width - num_imgs * max_img_width) / (num_imgs - 1)

x_start = margin_horizontal
x_offset = 1 * cm  # Mover todo a la derecha para centrar mejor

# Fila 1
y1 = h - 4 * cm - max_img_width
for i, filename in enumerate(row1):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            img.save(img_path)
        
        aspect = img.height / img.width
        if aspect >= 1:
            img_height = max_img_width
            img_width = img_height / aspect
        else:
            img_width = max_img_width
            img_height = img_width * aspect
        
        x = x_start + i * (max_img_width + gap) + x_offset
        c.drawImage(img_path, x, y1 + (max_img_width - img_height)/2, width=img_width, height=img_height)

# Fila 2
y2 = y1 - max_img_width - 1.8 * cm
for i, filename in enumerate(row2):
    img_path = os.path.join(img_folder, filename)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        if img.width > img.height:
            img = img.rotate(90, expand=True)
            img.save(img_path)
        
        aspect = img.height / img.width
        if aspect >= 1:
            img_height = max_img_width
            img_width = img_height / aspect
        else:
            img_width = max_img_width
            img_height = img_width * aspect
        
        x = x_start + i * (max_img_width + gap) + x_offset
        c.drawImage(img_path, x, y2 + (max_img_width - img_height)/2, width=img_width, height=img_height)

In [54]:
c.save()
print(f"PDF generado con imágenes finales en: {output_pdf}")

PDF generado con imágenes finales en: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/output_fotos_finales.pdf


## Axuliar - nariz en imágenes

Lee el CSV con las coordenadas de nariz, dibuja un círculo rojo en cada imagen donde haya nariz detectada, y guarda una copia marcada en otra carpeta.

- Usa `ast.literal_eval` para convertir las coordenadas tipo string a lista.
- Dibuja un círculo rojo con `PIL.ImageDraw`.
- Guarda las nuevas imágenes con prefijo `marcada_`.

Solo marca si hay nariz válida y si la imagen existe.


In [ ]:
# Rutas
csv_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/data.csv"
img_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized"
output_folder = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/resized_marcadas"

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer CSV
df = pd.read_csv(csv_path)

# Iterar sobre las imágenes
for _, row in df.iterrows():
    nombre = row['nombre_archivo']
    nariz_raw = row['nariz']
    if pd.isna(nariz_raw):
        continue  # Saltar si no hay coordenada
    
    try:
        nariz = ast.literal_eval(nariz_raw)
        img_path = os.path.join(img_folder, nombre)
        if not os.path.exists(img_path):
            print(f"Imagen no encontrada: {nombre}")
            continue
        
        img = Image.open(img_path)
        draw = ImageDraw.Draw(img)

        # Dibuja un círculo rojo de radio 8 px
        r = 8
        x, y = nariz
        draw.ellipse((x - r, y - r, x + r, y + r), fill='red')

        # Guardar imagen modificada
        out_path = os.path.join(output_folder, f"marcada_{nombre}")
        img.save(out_path)
        print(f"Nariz marcada en: {nombre}")
    
    except Exception as e:
        print(f"Error con {nombre}: {e}")

## Reflexiones y posibles mejoras

Algunas ideas para optimizar y mejorar el flujo de trabajo actual:

- **Paralelización de llamadas a la API:**  
    Actualmente, las imágenes se procesan de forma secuencial, lo que puede ser muy lento si el número de fotos es grande. Implementar procesamiento en paralelo (por ejemplo, usando `concurrent.futures.ThreadPoolExecutor` o `asyncio` si la librería lo permite) podría reducir significativamente el tiempo total de espera.

- **Reducir tokens en las respuestas:**  
    El prompt puede modificarse para que la API devuelva códigos cortos en lugar de cadenas largas. Por ejemplo:
        - "Perfil izquierda" → "PI"
        - "Girado izquierda" → "GI"
        - "Frente" → "F"
        - "Girado derecha" → "GD"
        - "Perfil derecha" → "PD"
        - "Sin dentadura" → "SD"
        - "Con dentadura" → "CD"  
    Esto reduce el tamaño de la respuesta y, por lo tanto, el costo por token.

- **Manejo de valores nulos:**  
    En vez de usar "NA" para valores no detectados, utilizar `null` (en JSON) o `None` (en Python) facilita el análisis posterior y la integración con pandas.

- **Filtrado previo en el prompt:**  
    Se puede modificar el prompt para que la API omita imágenes que no sean rostros válidos o que no cumplan ciertos criterios, evitando así respuestas innecesarias y ahorrando tokens.

Implementar estas mejoras puede hacer el proceso más eficiente, económico y robusto.
```